# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model
- Compare and contrast model validation strategies

## Let's Get Started

We included the code to pre-process the Ames Housing dataset below. This is done for the sake of expediency, although it may result in data leakage and therefore overly optimistic model metrics.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

## Train-Test Split

Perform a train-test split with a test set of 20% and a random state of 4.

In [2]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split


In [3]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

### Fit a Model

Fit a linear regression model on the training set

In [4]:
# Import LinearRegression from sklearn.linear_model
from sklearn.linear_model import LinearRegression

In [5]:
# Instantiate and fit a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

### Calculate MSE

Calculate the mean squared error on the test set

In [6]:
# Import mean_squared_error from sklearn.metrics
from sklearn.metrics import mean_squared_error

In [7]:
# Calculate MSE on test set
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.4f}')

Mean Squared Error: 0.1523


## Cross-Validation using Scikit-Learn

Now let's compare that single test MSE to a cross-validated test MSE.

In [8]:
# Import cross_val_score from sklearn.model_selection
from sklearn.model_selection import cross_val_score

In [9]:
# Find MSE scores for a 5-fold cross-validation
mse_scores = -cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')

In [10]:
# Get the average MSE score
average_mse = mse_scores.mean()
print(f'Average MSE from 5-fold cross-validation: {average_mse:.4f}')


Average MSE from 5-fold cross-validation: 0.1770


Compare and contrast the results. What is the difference between the train-test split and cross-validation results? Do you "trust" one more than the other?

In [ ]:
# Your answer here
# Train-Test Split: A single train-test split provides one estimate of the model's performance on unseen data. The data is divided into two sets: the training set used to train the model, and the test set used to evaluate it. The Mean Squared Error (MSE) is calculated on this single test set.
# Cross-Validation: Cross-validation, specifically k-fold cross-validation (with k=5 in this case), provides a more robust estimate of the model's performance. It divides the data into k "folds." The model is trained on k-1 of these folds and tested on the remaining fold. This process is repeated k times, with each fold serving as the test set once. The performance metric (MSE) is calculated for each of the k trials, and the average of these MSE values is taken as the final estimate of the model's performance.

## Level Up: Let's Build It from Scratch!

### Create a Cross-Validation Function

Write a function `kfolds(data, k)` that splits a dataset into `k` evenly sized pieces. If the full dataset is not divisible by `k`, make the first few folds one larger then later ones.

For example, if you had this dataset:

In [11]:
example_data = pd.DataFrame({
    "color": ["red", "orange", "yellow", "green", "blue", "indigo", "violet"]
})
example_data

,color
0,red
1,orange
2,yellow
3,green
4,blue
5,indigo
6,violet


`kfolds(example_data, 3)` should return:

* a dataframe with `red`, `orange`, `yellow`
* a dataframe with `green`, `blue`
* a dataframe with `indigo`, `violet`

Because the example dataframe has 7 records, which is not evenly divisible by 3, so the "leftover" 1 record extends the length of the first dataframe.

In [12]:
def kfolds(data, k):
    folds = []
    
    # Your code here
    n = len(data)
    # Integer division to get the base fold size
    fold_size = n // k 

    # Number of extra records to distribute 
    remainder = n % k

    start = 0
    for i in range(k):
        # Determine the size of this fold.  The first 'remainder' folds
        # get one extra record each.
        end = start + fold_size + (1 if i < remainder else 0)
        
        # Use .copy() to avoid chained indexing issues
        fold = data.iloc[start:end].copy() 
        folds.append(fold)
        start = end 
    
    return folds

In [13]:
results = kfolds(example_data, 3)
for result in results:
    print(result, "\n")

    color
0     red
1  orange
2  yellow 

   color
3  green
4   blue 

    color
5  indigo
6  violet 



### Apply Your Function to the Ames Housing Data

Get folds for both `X` and `y`.

In [15]:
# Apply kfolds() to ames_data with 5 folds
X_folds = kfolds(X, 5)
y_folds = kfolds(y, 5)


### Perform a Linear Regression for Each Fold and Calculate the Test Error

Remember that for each fold you will need to concatenate all but one of the folds to represent the training data, while the one remaining fold represents the test data.

In [ ]:
# Replace None with appropriate code
test_errs = []
k = 5

for n in range(k):
    # Split into train and test for the fold
    X_train = None
    X_test = None
    y_train = None
    y_test = None
    
    # Fit a linear regression model
    None
    
    # Evaluate test errors
    None

print(test_errs)

If your code was written correctly, these should be the same errors as scikit-learn produced with `cross_val_score` (within rounding error). Test this out below:

In [16]:
# Compare your results with sklearn results
model = LinearRegression()

# sklearn's cross_val_score
sklearn_mse_scores = -cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
sklearn_avg_mse = sklearn_mse_scores.mean()
print(f"Scikit-learn's average MSE: {sklearn_avg_mse:.4f}")

# Your kfolds and manual calculation
manual_mse_scores = []
for i in range(5):
    X_train_folds = [X_folds[j] for j in range(5) if j != i]
    X_test_fold = X_folds[i]
    y_train_folds = [y_folds[j] for j in range(5) if j != i]
    y_test_fold = y_folds[i]
    
    X_train_combined = pd.concat(X_train_folds)
    y_train_combined = pd.concat(y_train_folds)

    model.fit(X_train_combined, y_train_combined)
    y_pred_fold = model.predict(X_test_fold)
    mse_fold = mean_squared_error(y_test_fold, y_pred_fold)
    manual_mse_scores.append(mse_fold)

manual_avg_mse = np.mean(manual_mse_scores)
print(f"Your kfolds average MSE: {manual_avg_mse:.4f}")

print(f"Difference: {sklearn_avg_mse - manual_avg_mse:.6f}")


Scikit-learn's average MSE: 0.1770
Your kfolds average MSE: 0.1770
Difference: 0.000000


This was a bit of work! Hopefully you have a clearer understanding of the underlying logic for cross-validation if you attempted this exercise.

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are more robust than train-test split.